In [101]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os
import numpy as np
import time

In [141]:
startTime = time.time()

transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])

#train_data_path = 'relative path of training data set'
train_data_path = './horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1027, shuffle=False, num_workers=1)  


validation_data_path = './horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=256, shuffle=False, num_workers=1) 

for i, data in enumerate(trainloader):
    
    # inputs is the image
    # labels is the class of the image
    trnInputs, trnLabels = data

    # if you don't change the image size, it will be [batch_size, 1, 100, 100]
    # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
    
    #make inputs [1027, 1, 100, 100] to [10000,1027]
    trnInputs = trnInputs.view(1027, 10000)
    trnInputs = trnInputs.T


# load validation images of the batch size for every iteration
for i, data in enumerate(valloader):

    # inputs is the image
    # labels is the class of the image
    valInputs, valLabels = data

    # if you don't change the image size, it will be [batch_size, 1, 100, 100]
    # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
    #make [256, 1, 100, 100] to [10000,256]
    valInputs = valInputs.view(256, 10000)
    valInputs = valInputs.T
    
endTime = time.time()

print("Loading time : ", endTime - startTime)
print(trnLabels.shape)
print(trnInputs.shape)
print(valInputs.shape)

Loading time :  1.8834950923919678
torch.Size([1027])
torch.Size([10000, 1027])
torch.Size([10000, 256])


In [151]:
firstNodesNum = 4
secondNodesNum = 3
thirdNodesNum = 1

#init W which contains(w_1, w_2, w_3)
W = list()

w = torch.rand(firstNodesNum,10000)
W.append(w)

w = torch.rand(secondNodesNum, firstNodesNum)
W.append(w)

w = torch.rand(thirdNodesNum, secondNodesNum)
W.append(w)

#init A which contains(a_0, a_1, a_2, a_3)
A = [None] * 4
A[0] = trnInputs

#init B which contatins(b_1, b_2, b_3)
B = list()

b = torch.zeros(firstNodesNum, 1027)
B.append(b)

b = torch.zeros(secondNodesNum, 1027)
B.append(b)

b = torch.zeros(thirdNodesNum, 1027)
B.append(b)

#init Z which contains(z_1, z_2, z_3)
Z = [None] * 3

#init DW which contains(dw_1, dw_2, dw_3)
DW = [None] * 3

#init DB which contains(db_1, db_2, db_3)
DB = [None] * 3

#init DZ which contains(dz_1, dz_2, dz_3)
DZ = [None] * 3

#init DA which contains(da_1, da_2, da_3)
DA = [None] * 3


#print(W, A, B, Z, DW, DB, DZ, DA)

[tensor([[0.0264, 0.8154, 0.8748,  ..., 0.6699, 0.0034, 0.1542],
        [0.3781, 0.0027, 0.6471,  ..., 0.4163, 0.6371, 0.2404],
        [0.5707, 0.2952, 0.4838,  ..., 0.8571, 0.5467, 0.6763],
        [0.1958, 0.7025, 0.0894,  ..., 0.2580, 0.2490, 0.1291]]), tensor([[0.8597, 0.6264, 0.4496, 0.1106],
        [0.9409, 0.1387, 0.4972, 0.3197],
        [0.3858, 0.8897, 0.5442, 0.0726]]), tensor([[0.7112, 0.5790, 0.0749]])] [tensor([[0.5961, 0.6000, 0.6157,  ..., 0.3569, 0.3529, 0.3529],
        [0.5961, 0.5961, 0.6157,  ..., 0.3569, 0.3529, 0.3490],
        [0.5961, 0.6000, 0.6157,  ..., 0.3529, 0.3490, 0.3490],
        ...,
        [0.8941, 0.8078, 0.6627,  ..., 0.4627, 0.4235, 0.4431],
        [0.9020, 0.7922, 0.6627,  ..., 0.4627, 0.3961, 0.4157],
        [0.8745, 0.7529, 0.6980,  ..., 0.2353, 0.3882, 0.4941]]), None, None, None] [tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]

In [150]:


#forwardPass
for idx in range(3):
    Z[idx] = np.dot(W[idx], A[idx]) + B[idx]
    A[idx+1] = sigmoid(Z[idx])

#calculate loss
trainLoss = (-trnLabels * np.log(A[3]) - (1-trnLabels)*np.log(A[3])).sum



#calculate acc
#backPropagation
#update()

'''
#Training

calculateForwardPass()
backPropagation()
update()
calculateLoss
calculateAcc

#Validation
calculateForwardPass()
calculateLoss()
calculateAcc()
'''

[tensor([[0.2622, 0.0510, 0.2178,  ..., 0.5407, 0.9399, 0.4155],
        [0.6988, 0.0221, 0.7036,  ..., 0.8712, 0.9579, 0.8690],
        [0.3981, 0.1598, 0.6024,  ..., 0.3411, 0.5992, 0.4629],
        [0.3967, 0.2124, 0.2802,  ..., 0.8544, 0.4297, 0.6073]]), tensor([[0.9200, 0.1609, 0.2330, 0.2345],
        [0.2695, 0.8359, 0.6064, 0.6175],
        [0.0941, 0.7410, 0.5379, 0.3009]]), tensor([[0.8061, 0.5753, 0.7424]])] [tensor([[0.5961, 0.6000, 0.6157,  ..., 0.3569, 0.3529, 0.3529],
        [0.5961, 0.5961, 0.6157,  ..., 0.3569, 0.3529, 0.3490],
        [0.5961, 0.6000, 0.6157,  ..., 0.3529, 0.3490, 0.3490],
        ...,
        [0.8941, 0.8078, 0.6627,  ..., 0.4627, 0.4235, 0.4431],
        [0.9020, 0.7922, 0.6627,  ..., 0.4627, 0.3961, 0.4157],
        [0.8745, 0.7529, 0.6980,  ..., 0.2353, 0.3882, 0.4941]]), None, None, None] [tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]

TypeError: add(): argument 'other' (position 1) must be Tensor, not numpy.ndarray

In [124]:
def mul(w,x,z):
    z = np.dot(w,x)


In [126]:
w = torch.tensor([[1,2]])
x = torch.tensor([[3],
           [4]])

mul(w,x,&q)
print(q)

SyntaxError: invalid syntax (<ipython-input-126-ad6342d6593f>, line 5)

In [130]:
a = list()
b = [[1,2,3]]
c = [[4,5,6]]
a.append(b)
a.append(c)
print(a)
a[0] = [[6,7,8]]
print(a)

[[[1, 2, 3]], [[4, 5, 6]]]
[[[6, 7, 8]], [[4, 5, 6]]]


In [133]:
a = torch.rand(1,100)
print(a)

tensor([[0.8435, 0.3219, 0.2608, 0.1333, 0.4563, 0.4392, 0.8957, 0.2124, 0.8630,
         0.9576, 0.0384, 0.5711, 0.0071, 0.1401, 0.9306, 0.2092, 0.3163, 0.3397,
         0.7583, 0.8490, 0.0405, 0.3740, 0.6274, 0.9405, 0.9360, 0.9934, 0.3995,
         0.3997, 0.6955, 0.2007, 0.3166, 0.3688, 0.4869, 0.8656, 0.2430, 0.0601,
         0.8723, 0.1672, 0.9747, 0.5803, 0.0600, 0.2074, 0.4461, 0.6617, 0.8128,
         0.0347, 0.5991, 0.1257, 0.6689, 0.7075, 0.0838, 0.6281, 0.6497, 0.7052,
         0.5552, 0.0681, 0.3842, 0.3523, 0.7611, 0.0144, 0.0931, 0.6590, 0.4231,
         0.3908, 0.2922, 0.3065, 0.6952, 0.0174, 0.5757, 0.8114, 0.5712, 0.6518,
         0.8692, 0.2076, 0.6844, 0.0383, 0.0361, 0.9018, 0.7601, 0.2972, 0.2718,
         0.8246, 0.1904, 0.9240, 0.4888, 0.4807, 0.0984, 0.4154, 0.2688, 0.2576,
         0.7912, 0.8474, 0.7111, 0.8050, 0.8883, 0.6812, 0.1067, 0.3705, 0.1228,
         0.6804]])


In [139]:
a = list()
a.append(torch.rand(1,10))
print(a)
a[0] = torch.tensor([[2,3]])
print(a)

[tensor([[0.6959, 0.4416, 0.8918, 0.2744, 0.3413, 0.2304, 0.0848, 0.2467, 0.2599,
         0.7140]])]
[tensor([[2, 3]])]


In [140]:
A = 3
print(a)

[tensor([[2, 3]])]


In [145]:
a = [None]*3
a[0] = 3
a[1] = torch.tensor([2,3])
print(a)

[3, tensor([2, 3]), None]
